In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    """将表示为输入序列的图映射到隐藏向量的模块"""
    def __init__(self, input_dim, hidden_dim, use_cuda):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)  # LSTM层
        self.use_cuda = use_cuda  # 标志，指示是否使用CUDA进行计算

    def forward(self, x, hidden):
        output, hidden = self.lstm(x, hidden)  # LSTM前向传播
        return output, hidden  # 返回LSTM输出和隐藏状态

    def init_hidden(self, batch_size):
        """可训练的初始隐藏状态"""
        # 使用零初始化隐藏状态
        enc_init_hx = torch.zeros(1, batch_size, self.hidden_dim)
        enc_init_cx = torch.zeros(1, batch_size, self.hidden_dim)
        # 如果use_cuda为True，则将张量移动到CUDA设备
        if self.use_cuda:
            enc_init_hx = enc_init_hx.cuda()
            enc_init_cx = enc_init_cx.cuda()
        return (enc_init_hx, enc_init_cx)  # 返回初始化的隐藏状态作为元组

In [ ]:
class LocalAttention(nn.Module):
    """用于seq2seq解码器的通用局部注意力模块"""
    def __init__(self, dim, window_size, use_tanh=False, C=10, use_cuda=True):
        super(LocalAttention, self).__init__()
        self.use_tanh = use_tanh  # 是否使用tanh函数
        self.project_query = nn.Linear(dim, dim)  # 将查询向量投影到新的维度空间
        self.project_ref = nn.Conv1d(dim, dim, 1, 1)  # 将参考向量投影到新的维度空间
        self.C = C  # tanh探索
        self.tanh = nn.Tanh()  # tanh激活函数
        self.window_size = window_size  # 局部窗口大小
        
        v = torch.FloatTensor(dim)  # 创建一个张量v
        if use_cuda:
            v = v.cuda()  # 如果使用CUDA，将张量移动到CUDA设备
        self.v = nn.Parameter(v)  # 将张量v包装成可学习的参数
        self.v.data.uniform_(-(1. / math.sqrt(dim)), 1. / math.sqrt(dim))  # 对参数进行均匀初始化
        
    def forward(self, query, ref):
        """
        Args: 
            query: 当前时间步解码器的隐藏状态。大小为 batch x dim
            ref: 编码器的一组隐藏状态。大小为 sourceL x batch x hidden_dim
        """
        # ref 现在是 [batch_size x hidden_dim x sourceL] 的张量
        ref = ref.permute(1, 2, 0)  # 调整张量维度顺序
        q = self.project_query(query).unsqueeze(2)  # batch x dim x 1
        e = self.project_ref(ref)  # batch_size x hidden_dim x sourceL 
        
        # 确定窗口边界，注意力集中在中心向左延申一半到窗口内小于sourceL的长度
        start = max(0, i - self.window_size // 2)
        end = min(sourceL, start + self.window_size)
        
        # 将查询张量扩展到 window_size，为了可以与ref逐元素操作
        expanded_q = q.repeat(1, 1, end - start)  # batch x dim x window_size
        
        # 瞥见函数计算注意力权重
        u = torch.bmm(self.v.unsqueeze(0).expand(expanded_q.size(0), -1, -1),
                      self.tanh(expanded_q + e[:, :, start:end]))
        if self.use_tanh:
            logits = self.C * self.tanh(u.squeeze(1))
        else:
            logits = u.squeeze(1)
            
        return e, logits  # 返回注意力权重和logits


In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 embedding_dim,
                 hidden_dim,
                 max_length,
                 tanh_exploration,
                 terminating_symbol,
                 use_tanh,
                 decode_type,
                 n_glimpses=1,
                 beam_size=0,
                 use_cuda=True):
        super(Decoder, self).__init__()
        
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_glimpses = n_glimpses
        self.max_length = max_length
        self.terminating_symbol = terminating_symbol 
        self.decode_type = decode_type
        self.beam_size = beam_size
        self.use_cuda = use_cuda

        # 定义输入权重和隐藏权重的线性变换
        self.input_weights = nn.Linear(embedding_dim, 4 * hidden_dim)
        self.hidden_weights = nn.Linear(hidden_dim, 4 * hidden_dim)

        # 定义注意力机制
        self.pointer = LocalAttention(hidden_dim, use_tanh=use_tanh, C=tanh_exploration, use_cuda=self.use_cuda)
        self.glimpse = LocalAttention(hidden_dim, use_tanh=False, use_cuda=self.use_cuda)
        self.sm = nn.Softmax()

    # 生成掩码
    def apply_mask_to_logits(self, step, logits, mask, prev_idxs):
        # 如果没有提供掩码，则创建一个形状与logits相同的全零张量作为掩码
        if mask is None:
            mask = torch.zeros(logits.size()).byte()
            if self.use_cuda:
                mask = mask.cuda()

        # 克隆掩码，以便修改不影响原始掩码
        mask_clone = mask.clone()

        # 防止已经选择的索引再次被选择，或者允许重新选择并在目标函数中进行惩罚
        if prev_idxs is not None:
            # 将最近选择的符号的索引位置设为1（标记为已选择）
            # prev_idxs 是先前已经选择的符号的索引
            # 在掩码中标记已选择的位置，避免它们再次被选择
            mask_clone[[x for x in range(logits.size(0))], prev_idxs.data] = 1

            # 将已经选择的位置在logits中置为负无穷
            # 这样可以确保已选择的符号不会再次被选择
            logits[mask_clone] = -np.inf

        # 返回修改后的logits和掩码
        return logits, mask_clone


    def forward(self, decoder_input, embedded_inputs, hidden, context):
        """
        Args:
            decoder_input: 解码器的初始输入，大小为 [batch_size x embedding_dim]，可训练参数。
            embedded_inputs: 编码器输出的嵌入，大小为 [sourceL x batch_size x embedding_dim]
            hidden: 前一个隐藏状态，大小为 [batch_size x hidden_dim]，初始设置为 (enc_h[-1], enc_c[-1])
            context: 编码器输出，大小为 [sourceL x batch_size x hidden_dim] 
        """
        def recurrence(x, hidden, logit_mask, prev_idxs, step):
            
            hx, cx = hidden  # batch_size x hidden_dim
            
            # 计算门控信息
            gates = self.input_weights(x) + self.hidden_weights(hx)
            ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
            
            ingate = F.sigmoid(ingate)
            forgetgate = F.sigmoid(forgetgate)
            cellgate = F.tanh(cellgate)
            outgate = F.sigmoid(outgate)
            # 当前状态
            cy = (forgetgate * cx) + (ingate * cellgate)
            # 当前隐藏状态
            hy = outgate * F.tanh(cy)  # batch_size x hidden_dim

            #输出隐藏状态
            g_l = hy
            for i in range(self.n_glimpses):
                #获取ref和对应的logit
                ref, logits = self.glimpse(g_l, context)
                #应用掩码，确保没有被选中
                logits, logit_mask = self.apply_mask_to_logits(step, logits, logit_mask, prev_idxs)
                # [batch_size x h_dim x sourceL] * [batch_size x sourceL x 1] = 
                # [batch_size x h_dim x 1]
                #计算加权和，获得注意力加权后的ref，更新g_l，维度转换
                g_l = torch.bmm(ref, self.sm(logits).unsqueeze(2)).squeeze(2) 
                
            # 用指针把g_l和context（上下文）
            _, logits = self.pointer(g_l, context)
            
            # 在后续的循环中继续使用这些更新后的 logits 和掩码
            logits, logit_mask = self.apply_mask_to_logits(step, logits, logit_mask, prev_idxs)
            # 用softmax来看logit的概率分布
            probs = self.sm(logits)
            return hy, cy, probs, logit_mask

        
        batch_size = context.size(1)
        outputs = []
        selections = []
        steps = range(self.max_length)  # 或者直到终止符号？
        inps = []
        idxs = None
        mask = None

        #随机解码
        if self.decode_type == "stochastic":
            for i in steps:
                hx, cx, probs, mask = recurrence(decoder_input, hidden, mask, idxs, i)
                hidden = (hx, cx)
                # 从嵌入向量embedded_inputs中根据probs选择解码器的下一个输入 [batch_size x hidden_dim]
                decoder_input, idxs = self.decode_stochastic(
                    probs,
                    embedded_inputs,
                    selections)
                #跟踪解码器的输入
                inps.append(decoder_input)
                #生成输出序列，用beam搜索
                #大于1，保留样本中最大的
                if self.beam_size > 1:
                    outputs.append(probs[:, 0,:])
                else:
                    #去除第一个维度为 1 的维度， (1, batch_size, sourceL) 转换为 (batch_size, sourceL)
                    outputs.append(probs.squeeze(0))
                # Check for indexing
                selections.append(idxs)
                 # Should be done decoding
                
                if len(active) == 0:
                    break
                decoder_input = Variable(decoder_input.data.repeat(self.beam_size, 1))

            return (outputs, selections), hidden
    #在selection中根据probs选
    def decode_stochastic(self, probs, embedded_inputs, selections):
        """
        通过选择与最大输出对应的输入来为解码器生成下一个输入

        Args: 
            probs: [batch_size x sourceL]，概率分布张量
            embedded_inputs: [sourceL x batch_size x embedding_dim]，嵌入的输入张量
            selections: 在解码过程中先前选择的所有索引的列表
       Returns:
            大小为[batch_size x sourceL]的张量，包含对应于此次解码迭代中选择的[batch_size]个索引的输入的嵌入，以及相应的索引
        """
        batch_size = probs.size(0)
        # idxs 是[batch_size]的张量
        #从idx中多项式采样（蒙特卡洛采样）
        idxs = probs.multinomial().squeeze(1)

        # 避免已经选择过的索引被再次选择
        for old_idxs in selections:
            # 将新的 idxs 与先前的 idxs 逐元素比较。如果有任何匹配，
            # 则需要重新采样
            if old_idxs.eq(idxs).data.any():
                print(' [!] 由于竞争条件重新采样')
                idxs = probs.multinomial().squeeze(1)
                break

        # 从嵌入的输入张量中选择索引对应的嵌入
        sels = embedded_inputs[idxs.data, [i for i in range(batch_size)], :] 
        return sels, idxs


    def decode_beam(self, probs, embedded_inputs, beam, batch_size, n_best, step):
        active = []
        for b in range(batch_size):
            # 如果当前 beam[b] 已经完成，则跳过
            if beam[b].done:
                continue

            # 尝试对当前 beam[b] 进行推进，如果无法推进，则将其添加到 active 列表中
            if not beam[b].advance(probs.data[b]):
                active += [b]
        
        
        all_hyp, all_scores = [], []
        for b in range(batch_size):
            # 对每个 beam[b] 进行排序，得到最佳的 n_best 条路径及其对应的分数
            scores, ks = beam[b].sort_best()
            all_scores += [scores[:n_best]]
            # 根据索引 ks，获取每个最佳路径的假设
            hyps = zip(*[beam[b].get_hyp(k) for k in ks[:n_best]])
            all_hyp += [hyps]
        
        # 将所有最佳假设的索引组成一个张量
        all_idxs = Variable(torch.LongTensor([[x for x in hyp] for hyp in all_hyp]).squeeze())
      
        # 根据不同的维度情况选择合适的 idxs，确保最终选择的索引具有一致的形状
        if all_idxs.dim() == 2:
            if all_idxs.size(1) > n_best:
                idxs = all_idxs[:,-1]  # 选择每行的最后一个索引
            else:
                idxs = all_idxs
        elif all_idxs.dim() == 3:
            idxs = all_idxs[:, -1, :]  # 选择最后一维的索引
        else:
            if all_idxs.size(0) > 1:
                idxs = all_idxs[-1]  # 选择最后一个索引
            else:
                idxs = all_idxs
        
        # 如果使用 CUDA，则将 idxs 移动到 GPU 上
        if self.use_cuda:
            idxs = idxs.cuda()

        # 根据 idxs 从 embedded_inputs 中获取对应的嵌入向量，
        if idxs.dim() > 1:
            x = embedded_inputs[idxs.transpose(0,1).contiguous().data, 
                    [x for x in range(batch_size)], :]
        else:
            x = embedded_inputs[idxs.data, [x for x in range(batch_size)], :]

        # 将结果展平为二维张量，以及返回索引和活跃列表
        return x.view(idxs.size(0) * n_best, embedded_inputs.size(2)), idxs, active
